In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model, Sequential
import numpy as np 
import tensorflow.keras.utils as ku 

In [3]:
oov_token='OOV'
tokenizer =Tokenizer(oov_token=oov_token)
data = open('/content/sample_data/shakespear_poetry.txt').read()
corpus = data.lower().split('\n')

In [4]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1

In [5]:
tokenizer.word_index

{'OOV': 1,
 'and': 2,
 'the': 3,
 'to': 4,
 'of': 5,
 'my': 6,
 'i': 7,
 'in': 8,
 'that': 9,
 'thy': 10,
 'thou': 11,
 'with': 12,
 'for': 13,
 'is': 14,
 'love': 15,
 'not': 16,
 'but': 17,
 'a': 18,
 'me': 19,
 'thee': 20,
 'so': 21,
 'be': 22,
 'as': 23,
 'all': 24,
 'you': 25,
 'his': 26,
 'which': 27,
 'when': 28,
 'it': 29,
 'this': 30,
 'by': 31,
 'your': 32,
 'doth': 33,
 'do': 34,
 'from': 35,
 'on': 36,
 'or': 37,
 'no': 38,
 'then': 39,
 'have': 40,
 'what': 41,
 'are': 42,
 'if': 43,
 'more': 44,
 'mine': 45,
 'their': 46,
 'shall': 47,
 'sweet': 48,
 'time': 49,
 'will': 50,
 'they': 51,
 'beauty': 52,
 'nor': 53,
 'eyes': 54,
 'art': 55,
 'her': 56,
 'heart': 57,
 'yet': 58,
 'o': 59,
 'than': 60,
 'can': 61,
 'should': 62,
 'thine': 63,
 'now': 64,
 'where': 65,
 'make': 66,
 'one': 67,
 'hath': 68,
 'he': 69,
 'fair': 70,
 'still': 71,
 'how': 72,
 'eye': 73,
 'him': 74,
 'like': 75,
 'true': 76,
 'see': 77,
 'am': 78,
 'she': 79,
 'those': 80,
 'though': 81,
 'being':

In [6]:
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
input_sequences

[[35, 418],
 [35, 418, 878],
 [35, 418, 878, 167],
 [35, 418, 878, 167, 214],
 [35, 418, 878, 167, 214, 518],
 [9, 879],
 [9, 879, 135],
 [9, 879, 135, 352],
 [9, 879, 135, 352, 103],
 [9, 879, 135, 352, 103, 157],
 [9, 879, 135, 352, 103, 157, 200],
 [17, 23],
 [17, 23, 3],
 [17, 23, 3, 880],
 [17, 23, 3, 880, 62],
 [17, 23, 3, 880, 62, 31],
 [17, 23, 3, 880, 62, 31, 49],
 [17, 23, 3, 880, 62, 31, 49, 635],
 [26, 312],
 [26, 312, 636],
 [26, 312, 636, 103],
 [26, 312, 636, 103, 201],
 [26, 312, 636, 103, 201, 26],
 [26, 312, 636, 103, 201, 26, 279],
 [17, 11],
 [17, 11, 881],
 [17, 11, 881, 4],
 [17, 11, 881, 4, 63],
 [17, 11, 881, 4, 63, 86],
 [17, 11, 881, 4, 63, 86, 215],
 [17, 11, 881, 4, 63, 86, 215, 54],
 [1373, 10],
 [1373, 10, 1374],
 [1373, 10, 1374, 637],
 [1373, 10, 1374, 637, 12],
 [1373, 10, 1374, 637, 12, 123],
 [1373, 10, 1374, 637, 12, 123, 1375],
 [1373, 10, 1374, 637, 12, 123, 1375, 1376],
 [202, 18],
 [202, 18, 1377],
 [202, 18, 1377, 65],
 [202, 18, 1377, 65, 519],

In [8]:
max_length = max([len(size) for size in input_sequences])

input_sequences =np.array(pad_sequences(input_sequences,maxlen = max_length,padding = 'pre'))

In [9]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [ ]:
label[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
total_words

3212

In [10]:
model = Sequential()

    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
model.add(Embedding(total_words,100,input_length=max_length-1))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(total_words, activation='softmax'))  #(# A Dense Layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321200    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1000)              201000    
_________________________________________________________________
dense_1 (Dense)              (None, 3212)              3215212   
Total params: 3,898,212
Trainable params: 3,898,212
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
history = model.fit(predictors, label, epochs=50, verbose=1)

Epoch 1/50
484/484 [==============================] - 8s 16ms/step - loss: 2.5245 - accuracy: 0.4226
Epoch 2/50
484/484 [==============================] - 8s 16ms/step - loss: 2.0885 - accuracy: 0.5177
Epoch 3/50
484/484 [==============================] - 8s 16ms/step - loss: 1.7440 - accuracy: 0.5917
Epoch 4/50
484/484 [==============================] - 8s 16ms/step - loss: 1.4806 - accuracy: 0.6482
Epoch 5/50
484/484 [==============================] - 8s 16ms/step - loss: 1.2743 - accuracy: 0.6986
Epoch 6/50
484/484 [==============================] - 7s 15ms/step - loss: 1.1099 - accuracy: 0.7352
Epoch 7/50
484/484 [==============================] - 8s 16ms/step - loss: 0.9841 - accuracy: 0.7637
Epoch 8/50
484/484 [==============================] - 7s 15ms/step - loss: 0.8985 - accuracy: 0.7830
Epoch 9/50
484/484 [==============================] - 7s 15ms/step - loss: 0.8216 - accuracy: 0.7990
Epoch 10/50
484/484 [==============================] - 7s 15ms/step - loss: 0.7749 - accura

In [13]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
model.predict_classes()

AttributeError: ignored

In [35]:
seed_text = "Where all the treasure of thy lusty days"
next_words = 10
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
    #print('list ' ,token_list)
    predicted = np.argmax(model.predict(token_list, verbose=0),axis=1)
    #print('max predicted values',predicted)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        #print(index,predicted)
        if index == predicted:
            output_word = word
            print(word)
            break
    seed_text += " " + output_word
print(seed_text)

bear
held
groan
'
torn
torn
the
summer
of
them
Where all the treasure of thy lusty days bear held groan ' torn torn the summer of them
